### Function to differentiate stimulus types 

In [1]:
def return_X_sorted(HCP_DIR, subject, task_key):
    '''Loads _cor.txt and _err.txt files, concatenates and sorts after time
    
    Args:
    - HCP_DIR (str)  : path of directory 
    - subject (int)  : current subject 
    - task_key(str)  : task key of current task
    
    Returns: 
    - X (np.ndarray) : 80x2 ndarray with columns time stamp and errors (1 = error)
    '''
    corr = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/all_bk_cor.txt")
    err =  np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/all_bk_err.txt")
    corr = np.hstack((corr[:,0].reshape(-1,1), np.zeros_like(corr[:,0].reshape(-1,1))))
    err = np.hstack((err[:,0].reshape(-1,1), np.ones_like(err[:,0]).reshape(-1,1)))
    X = np.vstack((corr, err))
    X = X[X[:,0].argsort()]
    
    return X 

def diff_error_types(HCP_DIR, subjects):
    stimuli = ['faces', 'body', 'tools', 'places']
    task_keys = ["tfMRI_WM_RL", "tfMRI_WM_LR"]
    errortypes = ['0bk_', '2bk_']
    cue_duration = 2.565
    len_block = 10
    temp_zeros = np.zeros([80,4])
    for subject in subjects[:2]:
        for task_key in task_keys[:1]:
            X = return_X_sorted(HCP_DIR=HCP_DIR, subject=subject, task_key=task_key)

            for errortype in errortypes:
                temp_zeros = np.zeros([80,4])
                for index, stimulus in enumerate(stimuli):
                    stim_name = errortype + stimulus
                    stim = np.genfromtxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{stim_name}.txt")
                    #Search for index of start value (min distance between stim onset + 
                    # cue duration and start of block in onset times of X)
                    min_index = np.argmin(np.abs(X[:,0] - (stim[0] + cue_duration)))
                    # Create new column of zeros and ones 
                    temp_zeros[min_index:min_index+len_block, index] = 1
                    T = np.hstack((X[:,0].reshape(-1,1), temp_zeros))

                    #Create new file with time stamps 
                    M = np.hstack((X[min_index : min_index+len_block, 0].reshape(-1,1), np.full((len_block,1),2.5),
                                   np.full((len_block, 1), 1)))
                    # Optional if we want to include the first line 
                    M = np.vstack((stim, M))
                    # Write file in source directory
                    np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{stim_name}_ts.txt",
                               M, fmt=['%.3f','%.1f', '%.d'], delimiter='\t')
                
                #Write T into file
                np.savetxt(f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/all_{errortype}ts.txt",
                            T, fmt=['%.3f','%.d', '%.d','%.d', '%.d'], delimiter='\t')

diff_error_types(HCP_DIR, subjects)

NameError: name 'HCP_DIR' is not defined